# BART for Paraphrasing with Simple Transformers
#### Paraphrasing is the act of expressing something using different words while retaining the original meaning. Let’s see how we can do it with BART, a Sequence-to-Sequence Transformer Model.

### Introduction
BART is a denoising autoencoder for pretraining sequence-to-sequence models. BART is trained by (1) corrupting text with an arbitrary noising function, and (2) learning a model to reconstruct the original text.

BART uses a standard Transformer architecture (Encoder-Decoder) like the original Transformer model used for neural machine translation but also incorporates some changes from BERT (only uses the encoder) and GPT (only uses the decoder). You can refer to the 2.1 Architecture section of the BART paper for more details.

### Pre-Training BART
BART is pre-trained by minimizing the cross-entropy loss between the decoder output and the original sequence.
Masked Language Modeling (MLM)
MLM models such as BERT are pre-trained to predict masked tokens. This process can be broken down as follows:
Replace a random subset of the input with a mask token [MASK]. (Adding noise/corruption)
The model predicts the original tokens for each of the [MASK] tokens. (Denoising)
Importantly, BERT models can “see” the full input sequence (with some tokens replaced with [MASK]) when attempting to predict the original tokens. This makes BERT a bidirectional model, i.e. it can “see” the tokens before and after the masked tokens.


This is suited for tasks like classification where you can use information from the full sequence to perform the prediction. However, it is less suited for text generation tasks where the prediction depends only on the previous words.

### Autoregressive Models
Models used for text generation, such as GPT2, are pre-trained to predict the next token given the previous sequence of tokens. This pre-training objective results in models that are well-suited for text generation, but not for tasks like classification.


### BART Sequence-to-Sequence
BART has both an encoder (like BERT) and a decoder (like GPT), essentially getting the best of both worlds.
The encoder uses a denoising objective similar to BERT while the decoder attempts to reproduce the original sequence (autoencoder), token by token, using the previous (uncorrupted) tokens and the output from the encoder.


### A significant advantage of this setup is the unlimited flexibility of choosing the corruption scheme; including changing the length of the original input. Or, in fancier terms, the text can be corrupted with an arbitrary noising function.

The corruption schemes used in the paper are summarized below.
**Token Masking** — A random subset of the input is replaced with [MASK] tokens, like in BERT. <br>
**Token Deletion** — Random tokens are deleted from the input. The model must decide which positions are missing (as the tokens are simply deleted and not replaced with anything else).<br>
**Text Infilling** — A number of text spans (length can vary) are each replaced with a single [MASK] token.
Sentence Permutation — The input is split based on periods (.), and the sentences are shuffled.<br>
**Document Rotation** — A token is chosen at random, and the sequence is rotated so that it starts with the chosen token.<br>

The authors note that training BART with text infilling yields the most consistently strong performance across many tasks.
For the task we are interested in, namely paraphrasing, the pre-trained BART model can be fine-tuned directly using the input sequence (original phrase) and the target sequence (paraphrased sentence) as a Sequence-to-Sequence model.
This also works for tasks like summarization and abstractive question answering.

# Setup
We will use the Simple Transformers library, based on the Hugging Face Transformers library, to train the models.

In [2]:
!pip install simpletransformers




## Data Preparation
We will be combining three datasets to serve as training data for our BART Paraphrasing Model.
Google PAWS-Wiki Labeled (Final)
Quora Question Pairs Dataset
Microsoft Research Paraphrase Corpus (MSRP)

In [9]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir '/content/drive/MyDrive/END3/Session11_12'

mkdir: cannot create directory ‘/content/drive/MyDrive/END3/Session11_12’: File exists


In [4]:
!cd '/content/drive/MyDrive/END3/Session11_12'

In [6]:
!pwd

/content


In [7]:
!mkdir data
!wget https://storage.googleapis.com/paws/english/paws_wiki_labeled_final.tar.gz -P data
!tar -xvf data/paws_wiki_labeled_final.tar.gz -C data
!mv data/final/* data
!rm -r data/final

!wget http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv -P data

--2022-01-28 06:59:36--  https://storage.googleapis.com/paws/english/paws_wiki_labeled_final.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.128, 74.125.135.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4687157 (4.5M) [application/gzip]
Saving to: ‘data/paws_wiki_labeled_final.tar.gz’

paws_wiki_labeled_f 100%[===================>]   4.47M  --.-KB/s    in 0.02s   

2022-01-28 06:59:37 (231 MB/s) - ‘data/paws_wiki_labeled_final.tar.gz’ saved [4687157/4687157]

final/test.tsv
final/
final/train.tsv
final/dev.tsv
--2022-01-28 06:59:37--  http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv
Resolving qim.fs.quoracdn.net (qim.fs.quoracdn.net)... 151.101.1.2, 151.101.65.2, 151.101.129.2, ...
Connecting to qim.fs.quoracdn.net (qim.fs.quoracdn.net)|151.101.1.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length:

### Loading helper functions

In [8]:
import warnings

import pandas as pd


def load_data(file_path, input_text_column, 
              target_text_column, label_column, keep_label=1):
    df = pd.read_csv(file_path, sep="\t", error_bad_lines=False)
    df = df.loc[df[label_column] == keep_label]
    df = df.rename(
        columns={input_text_column: "input_text", target_text_column: "target_text"}
    )
    df = df[["input_text", "target_text"]]
    df["prefix"] = "paraphrase"

    return df


def clean_unnecessary_spaces(out_string):
    if not isinstance(out_string, str):
        warnings.warn(f">>> {out_string} <<< is not a string.")
        out_string = str(out_string)
    out_string = (
        out_string.replace(" .", ".")
        .replace(" ?", "?")
        .replace(" !", "!")
        .replace(" ,", ",")
        .replace(" ' ", "'")
        .replace(" n't", "n't")
        .replace(" 'm", "'m")
        .replace(" 's", "'s")
        .replace(" 've", "'ve")
        .replace(" 're", "'re")
    )
    return out_string

# Data Set - Display sample records

### Paraphrasing with BART
Once the data is prepared, training the model is quite simple.

In [10]:
import os
from datetime import datetime
import logging

import pandas as pd
from sklearn.model_selection import train_test_split
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs

#from utils import load_data, clean_unnecessary_spaces


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

In [11]:
# Google Data
train_df = pd.read_csv("data/train.tsv", sep="\t").astype(str)
eval_df = pd.read_csv("data/test.tsv", sep="\t").astype(str)

train_df = train_df.loc[train_df["label"] == "1"]
eval_df = eval_df.loc[eval_df["label"] == "1"]

train_df = train_df.rename(
    columns={"sentence1": "input_text", "sentence2": "target_text"}
)
eval_df = eval_df.rename(
    columns={"sentence1": "input_text", "sentence2": "target_text"}
)



In [12]:
train_df.head()

,id,input_text,target_text,label
1,2,The NBA season of 1975 -- 76 was the 30th seas...,The 1975 -- 76 season of the National Basketba...,1
3,4,When comparable rates of flow can be maintaine...,The results are high when comparable flow rate...,1
4,5,It is the seat of Zerendi District in Akmola R...,It is the seat of the district of Zerendi in A...,1
5,6,William Henry Henry Harman was born on 17 Febr...,"William Henry Harman was born in Waynesboro , ...",1
7,8,With a discrete amount of probabilities Formul...,Given a discrete set of probabilities formula ...,1


In [13]:
eval_df.head()

,id,input_text,target_text,label
2,3,"In January 2011 , the Deputy Secretary General...","In January 2011 , FIBA Asia deputy secretary g...",1
5,6,"During her sophomore , junior and senior summe...","During her second , junior and senior summers ...",1
7,8,"His father emigrated to Missouri in 1868 , but...",His father emigrated to Missouri in 1868 but r...,1
9,10,It is situated south of Köroğlu Mountains and ...,It is situated south of Köroğlu - mountains an...,1
10,11,The Río Blanco mine is a large copper mine loc...,The Río Blanco - Mine is a large copper mine i...,1


In [14]:
train_df = train_df[["input_text", "target_text"]]
eval_df = eval_df[["input_text", "target_text"]]

train_df["prefix"] = "paraphrase"
eval_df["prefix"] = "paraphrase"


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [15]:
train_df.head()

,input_text,target_text,prefix
1,The NBA season of 1975 -- 76 was the 30th seas...,The 1975 -- 76 season of the National Basketba...,paraphrase
3,When comparable rates of flow can be maintaine...,The results are high when comparable flow rate...,paraphrase
4,It is the seat of Zerendi District in Akmola R...,It is the seat of the district of Zerendi in A...,paraphrase
5,William Henry Henry Harman was born on 17 Febr...,"William Henry Harman was born in Waynesboro , ...",paraphrase
7,With a discrete amount of probabilities Formul...,Given a discrete set of probabilities formula ...,paraphrase


# MSRP Dataset:
https://datamelt.org/examples/data/msrp/msr_paraphrase_train.txt

In [ ]:
# MSRP Data
#train_df = pd.concat(
#    [
#        train_df,
#        load_data("msr_paraphrase_train.txt", "#1 String", "#2 String", "Quality"),
#    ]
#)
#eval_df = pd.concat(
#    [
#        eval_df,
#        load_data("msr_paraphrase_test.txt", "#1 String", "#2 String", "Quality"),
#    ]
#)


# Quora  Dataset

In [ ]:
# Quora Data

# The Quora Dataset is not separated into train/test, so we do it manually the first time.
#df = load_data(
#    "data/quora_duplicate_questions.tsv", "question1", "question2", "is_duplicate"
#)
#q_train, q_test = train_test_split(df)

#q_train.to_csv("data/quora_train.tsv", sep="\t")
#q_test.to_csv("data/quora_test.tsv", sep="\t")




INFO:numexpr.utils:NumExpr defaulting to 4 threads.


In [16]:
# The code block above only needs to be run once.
# After that, the two lines below are sufficient to load the Quora dataset.

# q_train = pd.read_csv("data/quora_train.tsv", sep="\t")
# q_test = pd.read_csv("data/quora_test.tsv", sep="\t")

#train_df = pd.concat([train_df, q_train])
#eval_df = pd.concat([eval_df, q_test])

train_df = train_df[["prefix", "input_text", "target_text"]]
eval_df = eval_df[["prefix", "input_text", "target_text"]]

train_df = train_df.dropna()
eval_df = eval_df.dropna()

train_df["input_text"] = train_df["input_text"].apply(clean_unnecessary_spaces)
train_df["target_text"] = train_df["target_text"].apply(clean_unnecessary_spaces)

eval_df["input_text"] = eval_df["input_text"].apply(clean_unnecessary_spaces)
eval_df["target_text"] = eval_df["target_text"].apply(clean_unnecessary_spaces)

print(train_df)


INFO:numexpr.utils:NumExpr defaulting to 4 threads.


           prefix  ...                                        target_text
1      paraphrase  ...  The 1975 -- 76 season of the National Basketba...
3      paraphrase  ...  The results are high when comparable flow rate...
4      paraphrase  ...  It is the seat of the district of Zerendi in A...
5      paraphrase  ...  William Henry Harman was born in Waynesboro, V...
7      paraphrase  ...  Given a discrete set of probabilities formula ...
...           ...  ...                                                ...
49384  paraphrase  ...  The Romance language currently spoken in Galic...
49390  paraphrase  ...  It is necessary to note that k is a vector con...
49393  paraphrase  ...  Tim Tim Henman won 6 -- 2, 7 -- 6 against Yevg...
49395  paraphrase  ...  He was considered an active member of the Coun...
49397  paraphrase  ...  She was at Cork on 24 June, and arrived in the...

[21829 rows x 3 columns]


Then, we set up the model and hyperparameter values. Note that we are using the pre-trained facebook/bart-large model, and fine-tuning it on our own dataset.

### Note: Below Seq2Seq model training was not successful with complete data set with insufficient memory :). Have taken smaller train and test set for model training.

In [17]:
train_df = train_df[0:10000]
eval_df = eval_df[0:1000]


In [20]:
print("train_df={}, eval_df={}".format(train_df.size,eval_df.size))

train_df=30000, eval_df=3000


In [21]:
import torch
torch.cuda.empty_cache()

In [22]:
import gc
gc.collect()

812

In [23]:
model_args = Seq2SeqArgs()
model_args.do_sample = True
model_args.eval_batch_size = 16
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 2500
model_args.evaluate_during_training_verbose = True
model_args.fp16 = False
model_args.learning_rate = 5e-5
model_args.max_length = 128
model_args.max_seq_length = 128
model_args.num_beams = None
model_args.num_return_sequences = 3
model_args.num_train_epochs = 2
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = False
model_args.save_steps = -1
model_args.top_k = 50
model_args.top_p = 0.95
model_args.train_batch_size = 8
model_args.use_multiprocessing = False
model_args.wandb_project = "Paraphrasing with BART"


model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-large",
    args=model_args,
)

model.train_model(train_df, eval_data=eval_df)

to_predict = [
    prefix + ": " + str(input_text)
    for prefix, input_text in zip(eval_df["prefix"].tolist(), eval_df["input_text"].tolist())
]
truth = eval_df["target_text"].tolist()

preds = model.predict(to_predict)

# Saving the predictions if needed
os.makedirs("predictions", exist_ok=True)

with open(f"predictions/predictions_{datetime.now()}.txt", "w") as f:
    for i, text in enumerate(eval_df["input_text"].tolist()):
        f.write(str(text) + "\n\n")

        f.write("Truth:\n")
        f.write(truth[i] + "\n\n")

        f.write("Prediction:\n")
        for pred in preds[i]:
            f.write(str(pred) + "\n")
        f.write(
            "________________________________________________________________________________\n"
        )


Downloading:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/971M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

...

INFO:simpletransformers.seq2seq.seq2seq_model:{'eval_loss': 0.5329207267080035}
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/best_model
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/checkpoint-2500-epoch-2
INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/1000 [00:00<?, ?it/s]

INFO:simpletransformers.seq2seq.seq2seq_model:{'eval_loss': 0.5329207267080035}
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/
INFO:simpletransformers.seq2seq.seq2seq_model: Training of facebook/bart-large model complete. Saved to outputs/.


Generating outputs:   0%|          | 0/63 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:2343: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


# Hyperparameters
The hyperparameter values are set to general, sensible values without doing hyperparameter optimization. For this task, the ground truth does not represent the only possible correct answer (nor is it necessarily the best answer). Because of this, tuning the hyperparameters to nudge the generated text as close to the ground truth as possible doesn’t make much sense.
Our aim is to generate good paraphrased sequences rather than to produce the exact paraphrased sequence from the dataset.

In [24]:
model_args.do_sample = True
model_args.num_beams = None
model_args.num_return_sequences = 3
model_args.max_length = 128
model_args.top_k = 50
model_args.top_p = 0.95

# Try out the model on your own sentences


In [28]:
import logging

from simpletransformers.seq2seq import Seq2SeqModel


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

model = Seq2SeqModel(
    encoder_decoder_type="bart", encoder_decoder_name="outputs"
)
'''
val_sentences = [          
          "A recording of folk songs done for the Columbia society in 1942 was largely arranged by Pjetër Dungu.",
          "In mathematical astronomy, his fame is due to the introduction of the astronomical globe, and his early contributions to understanding the movement of the planets.",
          "Why are people obsessed with Cara Delevingne?",
          "Earl St Vincent was a British ship that was captured in 1803 and became a French trade man.",
          "Worcester is a town and county city of Worcestershire in England.",
          "The goal of any Deep Learning model is to take in an input and generate the correct output."
]'''

val_sentences1 = [
"In April 2016 it was announced that Reid will also present the new sister show `` Extra Gear '' from `` Top Gear '' with Harris .	In April 2016 it was announced that Harris will also present the new sister broadcast `` Top Gear '' Extra Gear `` along with Reid .",
"The episode was written by Kirker Butler , and directed by Ron Rubio .	The consequence was directed by Ron Rubio and written by Kirker Butler .",
"In 2005 , the publication Richard Benton , Matthew Hyden and Steve Wilcox credited the Elite team in Volume 2-Issue 1 as Retro Gamer .	In 2005 , the release Retro Gamer the elite team in volume 2-issue 1 credited as Richard Benton , Matthew Hyden and Steve Wilcox .",
"The most promising individuals in each generation are terminated while the others are selected .	The most promising individuals in each generation are finished while the others are selected .",
"Flossach is a long river in Bavaria , Germany , a right-wing tributary of the Mindel and an indirect tributary of the Danube right .	Flossach is a river on the right in Bavaria , Germany , a long tributary of the Mindel and an indirect tributary of the Danube right .",
"After missing a week , the series returned on Monday 9 July at 10pm , cut from one hour to 30 minutes and reduced from six episodes to five .	The series returned on Monday , July 9 at 10 pm , after a missing week , from one hour to 30 minutes and from six episodes to five .",
"Nsanje is a district in the southern region of Nsanje The capital is Malawi The district includes an area of and has a population of 194,924 .	Nsanje is a district in the southern region of Malawi The capital is Nsanje The district consists of an area of and has a population of 194,924 .",
"The Dark Husband is a Big Finish Productions British drama based on the long-running audio science fiction television series `` Doctor Who '' .	The Dark Husband is a British drama of Big Finish Productions based on the long-running Science - Fiction - TV series '' Doctor Who `` .",
"The school was taken over in the 1960s by Patrick Kilroy , and his son Charles Kilroy took over the management of Kilroy 's College in 1999 .	The school was taken over by Patrick Kilroy in the 1960s . His son Charles Kilroy took over running of Kilroy 's College in 1999 .",
"The Green Park Stadium -- formerly known as the Modi Stadium -- is a cricket ground in Kanpur , India .	The Modi Stadium -- formerly known as Green Park Stadium -- is a cricket ground in Kanpur , India ."
]

for sentence in val_sentences1:
    original = sentence
    to_predict = [original]

    preds = model.predict(to_predict)

    print("---------------------------------------------------------")
    print(original)

    print()
    print("Predictions >>>")
    for pred in preds[0]:
        print(pred)

    print("---------------------------------------------------------")
    print()


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:2343: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


---------------------------------------------------------
In April 2016 it was announced that Reid will also present the new sister show `` Extra Gear '' from `` Top Gear '' with Harris .	In April 2016 it was announced that Harris will also present the new sister broadcast `` Top Gear '' Extra Gear `` along with Reid .

Predictions >>>
In April 2016, it was announced that Reid will also present the new sister show `` Extra Gear '' from `` Top Gear '' with Harris.In April 2017 it was also announced that Harris will present the New Sister Show '' Top Gear '', '' Extra Gear ''.
In April 2016, it was announced that Reid will also present the new sister show `` Extra Gear '' from `` Top Gear '' with Harris. In April 2016 it was also announced that Harris will present the New Sister Show '' Top Gear '', '' Extra Gear ''.
In April 2016, it was announced that Reid will also present the new sister show `` Extra Gear '' from `` Top Gear '' with Harris. In April 2016 it was also announced that Ha

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
The episode was written by Kirker Butler , and directed by Ron Rubio .	The consequence was directed by Ron Rubio and written by Kirker Butler .

Predictions >>>
The episode was written by Kirker Butler and directed by Ron Rubio. The consequence was written and produced by Kirkers Butler.
The episode was written by Kirker Butler and directed by Ron Rubio. The consequence was written and directed from Ron Rubio and was directed.
The episode was written by Kirker Butler and directed by Ron Rubio. The consequence was directed by Ronald Rubio and was written and written by Kirkker Butler.
---------------------------------------------------------



...

---------------------------------------------------------
The Dark Husband is a Big Finish Productions British drama based on the long-running audio science fiction television series `` Doctor Who '' .	The Dark Husband is a British drama of Big Finish Productions based on the long-running Science - Fiction - TV series '' Doctor Who `` .

Predictions >>>
The Dark Husband is a British drama by Big Finish Productions based on the long-running audio science fiction television series `` Doctor Who '' :
The Dark Husband is a British drama by Big Finish Productions based on the long-running audio science fiction television series `` Doctor Who ''.
The Dark Husband is a British drama by Big Finish Productions based on the long-running audio science fiction television series `` Doctor Who ''.
---------------------------------------------------------



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
The school was taken over in the 1960s by Patrick Kilroy , and his son Charles Kilroy took over the management of Kilroy 's College in 1999 .	The school was taken over by Patrick Kilroy in the 1960s . His son Charles Kilroy took over running of Kilroy 's College in 1999 .

Predictions >>>
The school was taken over by Patrick Kilroy in the 1960s, and his son Charles Kilroy took over the management of Kilroy's College in 1999.
The school was taken over by Patrick Kilroy in the 1960s, and his son Charles Kilroy took over the management of Kilroy's College in 1999.
The school was taken over by Patrick Kilroy in the 1960s, and his son Charles Kilroy took over the management of Kilroy's College in 1999.
---------------------------------------------------------



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
The Green Park Stadium -- formerly known as the Modi Stadium -- is a cricket ground in Kanpur , India .	The Modi Stadium -- formerly known as Green Park Stadium -- is a cricket ground in Kanpur , India .

Predictions >>>
The Green Park Stadium -- formerly known as the Modi Stadium -- is a cricket ground in Kanpur, India.
The Green Park Stadium -- formerly known as the Modi Stadium -- is a cricket ground in Kanpur, India.
The Green Park Stadium -- formerly known as the Modi Stadium -- is a cricket ground in Kanpur, India.
---------------------------------------------------------



### Results
Let’s look at some of the paraphrased sequences generated by the model for the test data. For each input sequence, the model will generate three ( num_return_sequences ) paraphrased sequences.